In [9]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [25]:
class PositionalEncoding(nn.Module):
    # 위치적 인코딩 할 것 
    # 실제 PE 를 구하는 식이 있음
    
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        # dropout 할 비율을 정하는 것
        pe = torch.zeros(max_len, d_model)
        # 0으로 이루어진 행렬을 max_len X d_model 형태로 만들어 pe 에 넣어둠
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # 위치는 0~ max_len 까지 원소를 만들고, max_len 길이의 행으로 변경
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # d_model 까지의 길이를 2 간격으로 숫자를 뽑아놓은 다음, 주어진 공식에 맞게 div_term 을 구함
        pe[:, 0::2] = torch.sin(position * div_term)
        # 짝수에는 sin()를, 홀수에는 cos() 함수를 취한다
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0,1)
        self.register_buffer('pe', pe)

In [22]:
class TransformerModel(nn.Module):
    
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout = 0.5):
        super(TransformerModel, self).__init__()
        # super(모델명, self).__init__() 으로 한번 명시해줘야 함
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layer = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)
        self.init_weights()
        
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0,1)
        # torch.ones(sz, sz) : 1로 구성된 행렬을 sz x sz 형태로 만듦
        # torch.ones(sz, sz) == 1 : 각 원소가 1이라면 True, 아니라면 False
        # torch.triu(torch.ones(sz, sz)) == 1 는 행렬의 윗 삼각형 부분만 건져냄(=true 로 만듦)
        # .transpose(0,1) : 행렬 변환
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        # 윗 삼각형은 -inf 로 만들고, 아랫삼각형은 0으로 채워둠
        return mask
    
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        # 인코더의 가중치 초기값은 -initrange ~ initrange
        self.decoder.bias.data.zeros_()
        # 디코더의 bias 는 0으로 초기화
        self.decoder.weight.data.uniform_(-initrange, initrange)
        # 디코더의 가중치 값도 동일하게 -initrange ~ initrange
        
    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            # mask 가 none 이거나, 길이가 맞지 않을 때
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask
        
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask) 
        # src 와 src_mask 를 인코더에 넣어 결과물을 받는다
        output = self.decoder(output) # 결과물을 디코더에 넣어 결과물을 얻는다
        return output

        
        

In [17]:
(torch.triu(torch.ones(3,2) == 1)).transpose(0,1)

tensor([[ True, False, False],
        [ True,  True, False]])

In [21]:
sz = 10
mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0,1)
# torch.ones(sz, sz) : 1로 구성된 행렬을 sz x sz 형태로 만듦
# torch.ones(sz, sz) == 1 : 각 원소가 1이라면 True, 아니라면 False
# torch.triu(torch.ones(sz, sz)) == 1 는 행렬의 윗 삼각형 부분만 건져냄(=true 로 만듦)
# .transpose(0,1) : 행렬 변환
print(mask)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
# 윗 삼각형은 -inf 로 만들고, 아랫삼각형은 0으로 채워둠
print(mask)

tensor([[ True, False, False, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False, False, False],
        [ True,  True,  True, False, False, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True]])
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
 

In [19]:
mask

tensor([[0.]])